In [20]:
import xarray as xr 
import numpy as np 

import iris
import efp_saffin as saff 
import datetime as dt 

import sys
sys.path.append('/home/users/cturrell/documents/eddy_feedback/') 
import functions.eddy_feedback as ef 

In [21]:
ds = xr.open_mfdataset('/gws/nopw/j04/arctic_connect/cturrell/reanalysis_data/SPARC_TEM_datasets/jra55_TEM_ubar_div1.nc',
                            parallel=True, chunks={'time': 31})

ds 

<xarray.Dataset>
Dimensions:   (time: 708, pressure: 22, latitude: 73)
Coordinates:
  * time      (time) datetime64[ns] 1958-01-14T09:00:00 ... 2016-12-14T09:00:00
  * pressure  (pressure) float32 1e+03 925.0 850.0 700.0 ... 5.0 3.0 2.0 1.0
  * latitude  (latitude) float32 -90.0 -87.5 -85.0 -82.5 ... 82.5 85.0 87.5 90.0
Data variables:
    ubar      (time, pressure, latitude) float32 dask.array<chunksize=(31, 22, 73), meta=np.ndarray>
    div1_pr   (time, pressure, latitude) float32 dask.array<chunksize=(31, 22, 73), meta=np.ndarray>
    div1_qg   (time, pressure, latitude) float32 dask.array<chunksize=(31, 22, 73), meta=np.ndarray>

In [22]:
# take seasonal average
ds = ds.resample(time='QS-DEC').mean('time')
ds = ds.sel(time=ds.time.dt.month.isin([12]))
ds = ds.where(ds.time.dt.year < 2016, drop=True)
ds = ds.sel(pressure=500.)
ds

<xarray.Dataset>
Dimensions:   (time: 59, latitude: 73)
Coordinates:
    pressure  float32 500.0
  * latitude  (latitude) float32 -90.0 -87.5 -85.0 -82.5 ... 82.5 85.0 87.5 90.0
  * time      (time) datetime64[ns] 1957-12-01 1958-12-01 ... 2015-12-01
Data variables:
    ubar      (time, latitude) float32 dask.array<chunksize=(6, 73), meta=np.ndarray>
    div1_pr   (time, latitude) float32 dask.array<chunksize=(6, 73), meta=np.ndarray>
    div1_qg   (time, latitude) float32 dask.array<chunksize=(6, 73), meta=np.ndarray>

In [23]:
ubar = ds.ubar
div1 = ds.div1_pr 

In [24]:
r = xr.corr(div1, ubar, dim='time')
r = r**2
r = r.sel( latitude=slice(25.,74.))
r

<xarray.DataArray (latitude: 20)>
dask.array<getitem, shape=(20,), dtype=float64, chunksize=(20,), chunktype=numpy.ndarray>
Coordinates:
    pressure  float32 500.0
  * latitude  (latitude) float32 25.0 27.5 30.0 32.5 ... 65.0 67.5 70.0 72.5

In [25]:
r.latitude

<xarray.DataArray 'latitude' (latitude: 20)>
array([25. , 27.5, 30. , 32.5, 35. , 37.5, 40. , 42.5, 45. , 47.5, 50. , 52.5,
       55. , 57.5, 60. , 62.5, 65. , 67.5, 70. , 72.5], dtype=float32)
Coordinates:
    pressure  float32 500.0
  * latitude  (latitude) float32 25.0 27.5 30.0 32.5 ... 65.0 67.5 70.0 72.5
Attributes:
    description:    Grid latitude
    standard_name:  latitude
    units:          degree_north

In [26]:
RR = r*np.cos( np.deg2rad(ds.latitude) )
R_weighted = RR.sum('latitude') / (np.cos( np.deg2rad(ds.latitude) )).sum('latitude')
R_weighted.values 

/home/users/cturrell/miniforge3/envs/eddy/lib/python3.12/site-packages/dask/array/numpy_compat.py:51: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


array(0.10629667)